In [24]:
from ultralytics import YOLO
import torch
import numpy as np
import os
import cv2
import yaml
from sklearn.model_selection import KFold
import pandas as pd
from collections import Counter
import shutil
from pathlib import Path
from ultralytics.utils.plotting import Annotator  # ultralytics.yolo.utils.plotting is deprecated

In [21]:
def create_folders(path, kfold_name, num_folds, classes):
    save_path = Path(path / f'{kfold_name}')
    save_path.mkdir(parents=True, exist_ok=True)

    ds_yamls = []
    for split in range(0, num_folds):
        # Create folder structure
        split_dir = save_path / f'split_{split}'
        split_dir.mkdir(parents=True, exist_ok=True)
        (split_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
        (split_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
        (split_dir / 'val' / 'images').mkdir(parents=True, exist_ok=True)
        (split_dir / 'val' / 'labels').mkdir(parents=True, exist_ok=True)

        # Create yaml config for models
        dataset_yaml = split_dir / f'split_{split}_dataset.yaml'
        ds_yamls.append(dataset_yaml)
    
        with open(dataset_yaml, 'w') as ds_y:
            yaml.safe_dump({
                'path': split_dir.as_posix(),
                'train': 'train',
                'val': 'val',
                'names': classes
            }, ds_y)
    return ds_yamls

def copy_images(images, labels, path, fold_nr, kfold_name, train_indices, test_indices):
    save_path = Path(path / f'{kfold_name}')

    # Copy training images
    for idx in train_indices:
        # destination paths for images and labels
        img_to_path = save_path / f'split_{fold_nr}' / 'train' / 'images'
        lbl_to_path = save_path / f'split_{fold_nr}' / 'train' / 'labels'

        if images[idx].exists() and labels[idx].exists():
            shutil.copy(images[idx], img_to_path / images[idx].name)
            shutil.copy(labels[idx], lbl_to_path / labels[idx].name)

    # Copy testing/validation images
    for idx in test_indices:
        # destination paths for images and labels
        img_to_path = save_path / f'split_{fold_nr}' / 'val' / 'images'
        lbl_to_path = save_path / f'split_{fold_nr}' / 'val' / 'labels'

        if images[idx].exists() and labels[idx].exists():
            shutil.copy(images[idx], img_to_path / images[idx].name)
            shutil.copy(labels[idx], lbl_to_path / labels[idx].name)

def consecutive_kfold(frames, num_folds):
    fold_size = len(frames) // num_folds

    for fold_idx in range(num_folds):
        start_idx = fold_idx * fold_size
        end_idx = (fold_idx + 1) * fold_size

        train_indices = np.arange(start_idx, end_idx)
        test_indices= np.concatenate([np.arange(0, start_idx), np.arange(end_idx, len(frames))])

        yield train_indices, test_indices

def save_output_video(fold_name, iteration, model):
    cap = cv2.VideoCapture("./datasets/data/scaled/vdo-scaled.mp4")
    fourcc = cv2.VideoWriter_fourcc('F','M','P','4')
    if iteration == -1:
        filename = f'./datasets/data/scaled/train/{fold_name}/pre-trained-output.avi'
    else:
        filename = f'./datasets/data/scaled/train/{fold_name}/split_{iteration}/output.avi'
    out = cv2.VideoWriter(filename=filename, fourcc=fourcc, fps=cap.get(cv2.CAP_PROP_FPS), frameSize=(int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        res = model(frame, device="mps")
        for r in res:
            # Annotate/Label bounding boxes
            annotator = Annotator(frame)
            boxes = r.boxes
            for box in boxes:
                b = box.xyxy[0]
                c = box.cls
                annotator.box_label(b, model.names[int(c)])
        frame = annotator.result()
        out.write(frame)
    cap.release()
    print(f"Finished parsing & saving video for {fold_name}/split_{iteration}")


In [23]:
# Get video result of pretrained model before KFold
cap = cv.VideoCapture("vdo.avi")
model = YOLO("yolov8n.pt")
fourcc = cv.VideoWriter_fourcc('F','M','P','4')
out = cv.VideoWriter(filename='./pretrained-output.avi', fourcc=fourcc, fps=cap.get(cv.CAP_PROP_FPS), frameSize=(int(cap.get(cv.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))))
save_output_video("No-fold", -1, model)


0: 384x640 4 cars, 1 truck, 26.9ms
Speed: 2.7ms preprocess, 26.9ms inference, 17.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 20.3ms
Speed: 2.0ms preprocess, 20.3ms inference, 15.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 18.4ms
Speed: 1.6ms preprocess, 18.4ms inference, 8.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 18.9ms
Speed: 1.5ms preprocess, 18.9ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 20.0ms
Speed: 1.3ms preprocess, 20.0ms inference, 7.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 truck, 17.9ms
Speed: 1.5ms preprocess, 17.9ms inference, 7.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 18.0ms
Speed: 1.4ms preprocess, 18.0ms inference, 13.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 truck, 17.4ms
Speed: 1.4ms preprocess, 17.4

In [ ]:
#Strategy A - consecutive images
# first 25% Training, rest 75% Testing
# included in first fold of Strategy B

In [ ]:
#Strategy B - consecutive images

# 1st fold
#  25%  |  25%  |  25%  |  25%
# Train   Test    Test    Test

# 2nd fold
#  25%  |  25%  |  25%  |  25%
#  Test   Train   Test    Test

# 3rd fold
#  25%  |  25%  |  25%  |  25%
#  Test    Test   Train    Test

# 4th fold
#  25%  |  25%  |  25%  |  25%
#  Test    Test    Test   Train

In [4]:
results_bk={}
dataset_path = Path('./data/scaled/train')

# Get classes
model = YOLO('yolov8n.pt', task='detect')
yaml_file = './data/scaled/data.yaml'
with open(yaml_file, 'r', encoding="utf8") as y:
    classes = yaml.safe_load(y)['names']
cls_idx = sorted([key for key, value in model.names.items() if value in classes])

# Get label filenames (labels are saved in txt files - one for each frame)
labels = sorted(dataset_path.glob("*labels/*.txt"))
indx = [l.stem for l in labels] # uses base filename as ID (no extension)

# Get source images (Path format)
supported_extensions = ['.jpg', '.jpeg', '.png']
images = []
for ext in supported_extensions:
    images.extend(sorted((dataset_path / 'images').rglob(f"*{ext}")))

# Create folder structure for folds ("split dataset" step #1)
ds_yamls = create_folders(dataset_path, 'B_KFold', 4, classes)

# Copy source images & labels to destination folder structure ("split dataset" step #2)
num_folds = 4
for fold_idx, (train_indices, test_indices) in enumerate(consecutive_kfold(indx, num_folds)):
    copy_images(images, labels, dataset_path, fold_idx, 'B_KFold', train_indices, test_indices)

# Train model on each dataset from the split
model = YOLO('yolov8n.pt', task='detect')
results_bk = {}
for k in range(num_folds):
    if k != 0:
        dataset_yaml = ds_yamls[k]
        model.train(data=dataset_yaml, epochs=25, batch=16, project=f'B_KFold_{k}')
        results_bk[k] = model.metrics
        save_output_video('B_KFold', k, model)
print(results_bk)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/beaerror/b-kfold/760c5990af68475897a96038eb77e086
COMET INFO:   Parameters:
COMET INFO:     batch_size : 16
COMET INFO:     epochs     : 25
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/Users/beatricepeptenaru/Projects/master

data/scaled/train/B_KFold/split_0/train/images
data/scaled/train/B_KFold/split_0/train/labels
data/scaled/train/B_KFold/split_0/train/images
data/scaled/train/B_KFold/split_0/train/labels
data/scaled/train/B_KFold/split_0/train/images
data/scaled/train/B_KFold/split_0/train/labels
data/scaled/train/B_KFold/split_0/train/images
data/scaled/train/B_KFold/split_0/train/labels
data/scaled/train/B_KFold/split_0/train/images
data/scaled/train/B_KFold/split_0/train/labels
data/scaled/train/B_KFold/split_0/train/images
data/scaled/train/B_KFold/split_0/train/labels
data/scaled/train/B_KFold/split_0/train/images
data/scaled/train/B_KFold/split_0/train/labels
data/scaled/train/B_KFold/split_0/train/images
data/scaled/train/B_KFold/split_0/train/labels
data/scaled/train/B_KFold/split_0/train/images
data/scaled/train/B_KFold/split_0/train/labels
data/scaled/train/B_KFold/split_0/train/images
data/scaled/train/B_KFold/split_0/train/labels
data/scaled/train/B_KFold/split_0/train/images
data/scaled/t

train: Scanning /Users/beatricepeptenaru/Projects/master/test/pythonProject/datasets/data/scal
val: Scanning /Users/beatricepeptenaru/Projects/master/test/pythonProject/datasets/data/scaled

Plotting labels to B_KFold_1/train2/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to B_KFold_1/train2
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

Optimizer stripped from B_KFold_3/train22/weights/last.pt, 6.2MB
Optimizer stripped from B_KFold_3/train22/weights/best.pt, 6.2MB

Validating B_KFold_3/train22/weights/best.pt...
Ultralytics YOLOv8.1.23 🚀 Python-3.9.6 torch-2.2.1 CPU (Apple M1 Pro)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%


                   all        233       2995      0.975       0.93      0.961      0.746
               bicycle        233         96      0.973      0.917      0.929      0.608
                   car        233       2794      0.953      0.981      0.971      0.815
                person        233        105          1      0.891      0.981      0.816
Speed: 0.7ms preprocess, 177.8ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to B_KFold_3/train22


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/beaerror/b-kfold-3/d67843fb4a284ca3b6b21ae5804a5ea3
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [11]                 : (30.126235961914062, 43.27528762817383)
COMET INFO:     lr/pg0 [51]               : (5.7159999999999995e-05, 0.00047996108799999997)
COMET INFO:     lr/pg1 [51]               : (5.7159999999999995e-05, 0.00047996108799999997)
COMET INFO:     lr/pg2 [51]               : (5.7159999999999995e-05, 0.00047996108799999997)
COMET INFO:     metrics/mAP50(B) [52]     : (0.94166, 0.96087)
COMET INFO:     metrics/mAP50-95(B) [52]  : (0.69065, 0.7460326027295832)
COMET INFO:     metrics/precision(B) [52] 


0: 384x640 8 cars, 56.3ms
Speed: 5.2ms preprocess, 56.3ms inference, 25.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 19.7ms
Speed: 1.2ms preprocess, 19.7ms inference, 8.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 19.8ms
Speed: 1.1ms preprocess, 19.8ms inference, 10.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 17.5ms
Speed: 1.3ms preprocess, 17.5ms inference, 15.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 16.6ms
Speed: 1.2ms preprocess, 16.6ms inference, 14.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 16.5ms
Speed: 1.1ms preprocess, 16.5ms inference, 14.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 16.6ms
Speed: 1.2ms preprocess, 16.6ms inference, 6.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 15.6ms
Speed: 1.3ms preprocess, 15.6ms inference, 6.6ms postprocess per image at shape (1, 3, 384, 640)

0:

In [ ]:
#Strategy C - random images
# 25% Train, 75% Test
# Code from YOLO Guidelines https://docs.ultralytics.com/guides/kfold-cross-validation/

In [202]:
results = {}
dataset_path = Path('./data/scaled/train')

# Get classes
model = YOLO('yolov8n.pt', task='detect')
yaml_file = './data/scaled/data.yaml'
with open(yaml_file, 'r', encoding="utf8") as y:
    classes = yaml.safe_load(y)['names']
cls_idx = sorted([key for key, value in model.names.items() if value in classes])

# Get label filenames (labels are saved in txt files - one for each frame)
labels = sorted(dataset_path.glob("*labels/*.txt"))
indx = [l.stem for l in labels] # uses base filename as ID (no extension)

# Label DataFrame for KFold
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)
for label in labels:
    lbl_counter = Counter()

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`

# KFold split
ksplit = 4
kf = KFold(n_splits=ksplit, shuffle=True, random_state=22)   # setting random_state for repeatable results
kfolds = list(kf.split(labels_df))
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (val, train) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[labels_df.iloc[val].index] = 'val'

fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

supported_extensions = ['.jpg', '.jpeg', '.png']

# Get source images (Path format)
images = []
for ext in supported_extensions:
    images.extend(sorted((dataset_path / 'images').glob(f"*{ext}")))

# Create folder structure for folds ("split dataset" step #1)
ds_yamls = create_folders(dataset_path, 'R_KFold', 4, classes)

# Copy source images & labels to destination folder structure ("split dataset" step #2)
for image, label in zip(images, labels):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / 'images'
        lbl_to_path = save_path / split / k_split / 'labels'

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

# Train model on each dataset from the split
model = YOLO('yolov8n.pt', task='detect')
results = {}
for k in range(ksplit):
    dataset_yaml = ds_yamls[k]
    model.train(data=dataset_yaml, epochs=25, batch=16, project=f'random_kfold_{k}')
    save_output_video('R_KFold', k, model)
    results[k] = model.metrics
    
print(results)

New https://pypi.org/project/ultralytics/8.1.24 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.23 🚀 Python-3.9.6 torch-2.2.1 CPU (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data/scaled/train/R-KFold/split_1/split_1_dataset.yaml, epochs=25, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=random_kfold_0, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=Non

train: Scanning /Users/beatricepeptenaru/Projects/master/test/pythonProject/datasets/data/scal

train: New cache created: /Users/beatricepeptenaru/Projects/master/test/pythonProject/datasets/data/scaled/train/R-KFold/split_1/train/labels.cache



val: Scanning /Users/beatricepeptenaru/Projects/master/test/pythonProject/datasets/data/scaled

val: New cache created: /Users/beatricepeptenaru/Projects/master/test/pythonProject/datasets/data/scaled/train/R-KFold/split_1/val/labels.cache


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%


                   all        233       3073      0.973      0.975      0.973      0.747
               bicycle        233        133      0.969       0.94      0.949      0.589
                   car        233       2800      0.962      0.992      0.976      0.857
                person        233        140      0.989      0.993      0.995      0.794
Speed: 0.7ms preprocess, 185.9ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to random_kfold_3/train8



[ERROR:0@34723.842] global cap.cpp:645 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:430: error: (-215:Assertion failed) !filename_pattern.empty() in function 'open'




0: 384x640 8 cars, 776.1ms
Speed: 8.0ms preprocess, 776.1ms inference, 15.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 19.3ms
Speed: 1.4ms preprocess, 19.3ms inference, 9.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 20.6ms
Speed: 1.7ms preprocess, 20.6ms inference, 11.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 20.0ms
Speed: 1.2ms preprocess, 20.0ms inference, 116.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 17.2ms
Speed: 1.0ms preprocess, 17.2ms inference, 45.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 16.2ms
Speed: 1.1ms preprocess, 16.2ms inference, 9.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 16.2ms
Speed: 1.1ms preprocess, 16.2ms inference, 7.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 17.1ms
Speed: 1.0ms preprocess, 17.1ms inference, 7.6ms postprocess per image at shape (1, 3, 384, 640)

0

NameError: name 'experiment' is not defined

In [16]:
# Redo results B_KFold
model = YOLO('./B_KFold_0/train8/weights/best.pt', task='detect')
save_output_video('B_KFold', 0, model)

model = YOLO('./B_KFold_1/train2/weights/best.pt', task='detect')
save_output_video('B_KFold', 1, model)

model = YOLO('./B_KFold_2/train2/weights/best.pt', task='detect')
save_output_video('B_KFold', 2, model)

model = YOLO('./B_KFold_3/train22/weights/best.pt', task='detect')
save_output_video('B_KFold', 3, model)


0: 384x640 9 cars, 23.7ms
Speed: 1.0ms preprocess, 23.7ms inference, 8.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 19.5ms
Speed: 1.9ms preprocess, 19.5ms inference, 18.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 19.3ms
Speed: 1.9ms preprocess, 19.3ms inference, 8.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 20.3ms
Speed: 1.7ms preprocess, 20.3ms inference, 16.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 17.1ms
Speed: 1.3ms preprocess, 17.1ms inference, 7.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 19.1ms
Speed: 1.3ms preprocess, 19.1ms inference, 14.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 17.9ms
Speed: 1.7ms preprocess, 17.9ms inference, 15.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 17.9ms
Speed: 1.4ms preprocess, 17.9ms inference, 7.7ms postprocess per image at shape (1, 3, 384, 640)

0: 

In [29]:
#Redo results R-KFold
model = YOLO('./random_kfold_0/train8/weights/best.pt', task='detect')
save_output_video('R-KFold', 1, model)

model = YOLO('./random_kfold_1/train8/weights/best.pt', task='detect')
save_output_video('R-KFold', 2, model)

model = YOLO('./random_kfold_2/train8/weights/best.pt', task='detect')
save_output_video('R-KFold', 3, model)

model = YOLO('./random_kfold_3/train8/weights/best.pt', task='detect')
save_output_video('R-KFold', 4, model)


0: 384x640 8 cars, 26.1ms
Speed: 1.8ms preprocess, 26.1ms inference, 20.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 19.6ms
Speed: 1.8ms preprocess, 19.6ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 19.4ms
Speed: 1.6ms preprocess, 19.4ms inference, 19.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 cars, 18.2ms
Speed: 1.3ms preprocess, 18.2ms inference, 17.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 19.0ms
Speed: 1.3ms preprocess, 19.0ms inference, 16.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 18.8ms
Speed: 1.6ms preprocess, 18.8ms inference, 17.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 19.1ms
Speed: 1.4ms preprocess, 19.1ms inference, 7.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 cars, 18.2ms
Speed: 1.4ms preprocess, 18.2ms inference, 7.7ms postprocess per image at shape (1, 3, 384, 640)

0: